## ReMKiT1D input generator - 2-fluid problem with no outflow boundary conditions - MMS

This notebook generates the Method of Manufactured Solutions test from the ReMKiT1D paper. See section 6.1.2. in that paper for more details.

The following are dependencies for this example

In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt
import panel as pn
import pickle

import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds
import RMK_support.common_models as cm
import RMK_support.sk_normalization as skn


### Some useful constants

In [ ]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
amu = 1.6605390666e-27 #atomic mass unit
ionMass = 2.014*amu # deuterium mass
epsilon0 = 8.854188e-12 #vacuum permittivity 
heavySpeciesMass = 2.014 #in amus

### Wrapper initialization

In [ ]:
rk = RKWrapper()

### Global parameters for IO files

In [ ]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_MMS_test/" 
rk.setHDF5Path(hdf5Filepath)

### Setting options for external libraries used by ReMKiT1D

#### MPI


In [ ]:
numProcsX = 8 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic 
numProcs = numProcsX * numProcsH
haloWidth = 1 # Halo width in cells

rk.setMPIData(numProcsX,numProcsH,haloWidth)

### Normalization

In [ ]:
rk.setNormDensity(1.0e19)
rk.setNormTemperature(10.0)
rk.setNormRefZ(1.0)

norms = skn.calculateNorms(10.0,1.0e19,1.0)

lengthNorm = norms["length"]

### Grid initialization

In [ ]:
L=10
Nx = 1024
xGridWidths = L/Nx*np.ones(Nx)
vGridWidths = np.ones(1)
lMax = 0
gridObj = Grid(xGridWidths,vGridWidths,lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True,isLengthInMeters=True)

In [ ]:
# Add the grid to the config file
rk.grid = gridObj

### Handling particle species data

In [ ]:
rk.addSpecies("e",0,atomicA=elMass/amu,charge=-1.0,associatedVars=["ne","Ge"]) 
rk.addSpecies("D+",-1,atomicA=2.014,charge=1.0,associatedVars=["ni","Gi"])

electronSpecies = rk.getSpecies("e")
ionSpecies = rk.getSpecies("D+")

#### Calculate variable initial values based on MMS test

In [ ]:
# Manufactured solution terms
dualGrid = gridObj.xGrid + gridObj.xWidths/2 
dn = 0.1
n = np.ones(Nx) + dn * (gridObj.xGrid-L)/L
ndual = np.ones(Nx) + dn * (dualGrid-L)/L

T0 = 0.5
u0 = 0.01

u = -u0 * gridObj.xGrid*(gridObj.xGrid-L)/L**2
udual = -u0 *  dualGrid*((dualGrid-L)/L)/L
dudx = -u0*(2*gridObj.xGrid-L)/L**2*lengthNorm
dudualdx = -u0*(2*dualGrid-L)/L**2*lengthNorm

gamma = n * u 
gammadual = ndual*udual

dndx = dn/L*lengthNorm

dGamma = dndx*u+dudx*n
dGammadual = dndx*udual + dudualdx*ndual
duGammadual = dGammadual*udual + dudualdx*gammadual

fixBoundaryCells = False
# Numerically modify duGammadual to account for fact ReMKiT1D uses extended boundary cells on staggered grid
if fixBoundaryCells:
    duGammadual[0] = (udual[0]+udual[1])*(gammadual[0]+gammadual[1])/4 *lengthNorm/ (xGridWidths[0]+xGridWidths[1]/2) 
    duGammadual[-2] = -(udual[-2]+udual[-3])*(gammadual[-2]+gammadual[-3])/4 *lengthNorm/ (xGridWidths[-1]+xGridWidths[-2]/2) 

T = T0*np.ones(Nx)

Edual = - (0.5 * T * dndx + duGammadual)/ndual # Using assumed normalization

In [ ]:
# Set conserved variables in container

rk.addVarAndDual("ne",n,units='$10^{19} m^{-3}$',isCommunicated=True) #Units are not used by ReMKiT1D, but are useful to specify for later plotting
rk.addVarAndDual("ni",n,units='$10^{19} m^{-3}$',isCommunicated=True)
rk.addVarAndDual("Ge",gammadual,primaryOnDualGrid=True,isCommunicated=True) # Ge_dual is evolved, and Ge is derived
rk.addVarAndDual("Gi",gammadual,primaryOnDualGrid=True,isCommunicated=True)

# Temperatures
rk.addVarAndDual("Te",T,isDerived=True,units='$10eV$',isCommunicated=True)


# Set E field

rk.addVarAndDual("E",Edual,primaryOnDualGrid=True)

# Set derived fluid quantities

rk.addVarAndDual("ue",isDerived=True,primaryOnDualGrid=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["Ge_dual","ne_dual"]),isCommunicated=True)
rk.addVarAndDual("ui",isDerived=True,primaryOnDualGrid=True,derivationRule=sc.derivationRule("flowSpeedFromFlux",["Gi_dual","ni_dual"]),isCommunicated=True)

# Set scalar quantities 
rk.addVar("time",isScalar=True,isDerived=True)


### Models 

### Density advection

In [ ]:
#Electron continuity advection

#Adding the model tag to tag list
modelTag = "continuity-ne"

#Initializing model using common models 
electronContModel = cm.staggeredAdvection(modelTag=modelTag, advectedVar="ne",
                                          fluxVar="Ge_dual", advectionSpeed="ue", rightOutflow=False)

rk.addModel(electronContModel.dict())

In [ ]:
#Ion continuity advection

#Adding the model tag to tag list
modelTag = "continuity-ni"

#Initializing model using common models
ionContModel = cm.staggeredAdvection(modelTag=modelTag, advectedVar="ni",
                                     fluxVar="Gi_dual", advectionSpeed="ui", rightOutflow=False)

rk.addModel(ionContModel.dict())

### Pressure gradient forces

In [ ]:
#Electron pressure grad

#Adding the model tag to tag list
modelTag = "pressureGrad-Ge"

#Initializing model
electronPressureGradModel = cm.staggeredPressureGrad(modelTag=modelTag,fluxVar="Ge_dual",densityVar="ne",temperatureVar="Te",speciesMass=elMass)

rk.addModel(electronPressureGradModel.dict())

In [ ]:
#Ion pressure grad

#Adding the model tag to tag list
modelTag = "pressureGrad-Gi"

#Initializing model
ionPressureGradModel = cm.staggeredPressureGrad(modelTag=modelTag,fluxVar="Gi_dual",densityVar="ni",temperatureVar="Te",speciesMass=ionMass)

rk.addModel(ionPressureGradModel.dict())

### Momentum advection

In [ ]:
#Electron momentum advection

#Adding the model tag to tag list
modelTag = "advection-Ge"

#Initializing model
electronMomAdvModel = cm.staggeredAdvection(modelTag=modelTag
                                        ,advectedVar="Ge_dual"
                                        ,fluxVar=""
                                        ,staggeredAdvectionSpeed="ue_dual"
                                        ,rightOutflow=False,
                                        staggeredAdvectedVar=True)

rk.addModel(electronMomAdvModel.dict())

In [ ]:
#Ion momentum advection

#Adding the model tag to tag list
modelTag = "advection-Gi"

#Initializing model
ionMomAdvModel = cm.staggeredAdvection(modelTag=modelTag
                                        ,advectedVar="Gi_dual"
                                        ,fluxVar=""
                                        ,staggeredAdvectionSpeed="ui_dual"
                                        ,rightOutflow=False,
                                        staggeredAdvectedVar=True)

rk.addModel(ionMomAdvModel.dict())

### Ampere-Maxwell term and Lorentz force

In [ ]:
#Ampere-Maxwell E field equation 
 
#Adding the model tag to tag list
modelTag = "ampereMaxwell"

#Initializing model
ampereMawellModel = cm.ampereMaxwell(modelTag=modelTag,
                                     eFieldName="E_dual",
                                     speciesFluxes=["Ge_dual","Gi_dual"],
                                     species=[electronSpecies,ionSpecies])

rk.addModel(ampereMawellModel.dict())

In [ ]:
#Lorentz force terms 
 
#Adding the model tag to tag list
modelTag = "lorentzForce"

#Initializing model
lorentzForceModel = cm.lorentzForces(modelTag=modelTag,
                                     eFieldName="E_dual",
                                     speciesFluxes=["Ge_dual","Gi_dual"],
                                     speciesDensities=["ne_dual","ni_dual"],
                                     species=[electronSpecies,ionSpecies])

rk.addModel(lorentzForceModel.dict())

### MMS Source model

In [ ]:
nSource = dGamma

gammaSourceion = duGammadual +  (0.5*T * dndx - ndual*Edual)*elMass/ionMass 

mmsModel = sc.CustomModel("mmsModel")

# Continuity equation MMS sources
mmsModel.addTerm("sourcene",cm.simpleSourceTerm("ne",nSource))
mmsModel.addTerm("sourceni",cm.simpleSourceTerm("ni",nSource))

# Momentum equation MMS source

vData = sc.VarData(reqRowVars=["ni_dual"],reqRowPowers=[-1.0])
    
sourceTermGammaion = sc.GeneralMatrixTerm("Gi_dual","ni",spatialProfile=gammaSourceion.tolist(),varData=vData,stencilData=sc.diagonalStencil())

mmsModel.addTerm("sourceGi",sourceTermGammaion)

rk.addModel(mmsModel.dict())


### Integrator options

In [ ]:
integrator = sc.picardBDEIntegrator(absTol=10.0,convergenceVars=["ne","ni","Ge_dual","Gi_dual"],nonlinTol=1e-14) 

rk.addIntegrator("BE",integrator)

### Timestep control

Here the timestep is rescaled based on collisionality (though this is not strictly necessary).

In [ ]:
initialTimestep=10.0

rk.setIntegratorGlobalData(3,2,initialTimestep) 

timestepControllerOptions = sc.scalingTimestepController(["ne","Te"],[-1.0,1.5])

rk.setTimestepController(timestepControllerOptions)

### Controlling integration steps

In [ ]:
bdeStep = sc.IntegrationStep("BE",defaultEvaluateGroups=[1,2,3],defaultUpdateModelData=True,defaultUpdateGroups=[1,2,3])

for tag in rk.modelTags():
    bdeStep.addModel(tag)

rk.addIntegrationStep("StepBDE",bdeStep.dict())

### Time loop options

Running for a set amount of normalized time:

In [ ]:
rk.setTimeTargetTimestepping(20000.0)
rk.setMinimumIntervalOutput(1000.0)

### Setting up diagnostic variable outputs

Using the wrapper feature that extracts all (non-generated) (model,term) pairs that evolve a given variable to produce diagnostic variables using manipulators. 

In [ ]:
terms = rk.getTermsThatEvolveVar("ne")

for pair in terms:
    model,term=pair
    rk.addVar(model+term,isDerived=True)
    rk.addManipulator(model+term,sc.termEvaluatorManipulator([pair],model+term))

In [ ]:
terms = rk.getTermsThatEvolveVar("Ge_dual")

for pair in terms:
    model,term=pair
    rk.addVar(model+term,isDerived=True)
    rk.addManipulator(model+term,sc.termEvaluatorManipulator([pair],model+term))

In [ ]:
terms = rk.getTermsThatEvolveVar("Gi_dual")

for pair in terms:
    model,term=pair
    rk.addVar(model+term,isDerived=True)
    rk.addManipulator(model+term,sc.termEvaluatorManipulator([pair],model+term))

In [ ]:
terms = rk.getTermsThatEvolveVar("E_dual")

for pair in terms:
    model,term=pair
    rk.addVar(model+term,isDerived=True)
    rk.addManipulator(model+term,sc.termEvaluatorManipulator([pair],model+term))

### Write config file

In [ ]:
rk.writeConfigFile()

### Data analysis

In [ ]:
numFiles = 20

In [ ]:
loadFilenames = [hdf5Filepath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]

In [ ]:
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

#### Explore data using basic dashboard

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,gridObj)

dashboard.fluid2Comparison().show()


### Generating plots for the paper

In [ ]:
varsToCheck = ['ne','ue_dual','E_dual']
mmsVars = [n,udual,Edual]

errors = [np.abs(loadedData[varsToCheck[i]]-mmsVars[i])/abs(mmsVars[i]) for i in range(len(mmsVars))]
maxErrors = [error.reduce(np.nanmax,'x') for error in errors]

In [ ]:
curves = hv.Overlay([hv.Curve(maxErrors[i],label=varsToCheck[i]) for i in range(len(mmsVars))])

In [ ]:
curves.opts(ylim=(1e-10,1),logy=True)

In [ ]:
maxErrorDict = {}
maxErrorDictFix = {}

#### Add maximum errors based on which run was performed

To reproduce the results, run ReMKiT1D with the appropriate fixBoundaryCells flag and the number of spatial cells Nx

In [ ]:
if fixBoundaryCells:
    maxErrorDictFix[Nx]=maxErrors
else:
    maxErrorDict[Nx]=maxErrors

In [ ]:
pickle.dump((maxErrorDict,maxErrorDictFix),open("mmsTest.pkl","wb"))

In [ ]:
maxErrorDict,maxErrorDictFix= pickle.load(open("mmsTest.pkl","rb"))

In [ ]:
plotNoFix = hv.Overlay([hv.Curve((list(maxErrorDict.keys()),[maxErrorDict[key][i][-1] for key in maxErrorDict.keys()])) for i in range(len(mmsVars))])
plotNoFix.opts(logx=True,logy=True,xlabel='$N_x$',ylabel='max($\delta$)',ylim=(1e-9,1))

In [ ]:
plotFix = hv.Overlay([hv.Curve((list(maxErrorDictFix.keys()),[maxErrorDictFix[key][i][-1] for key in maxErrorDictFix.keys()]),label=varsToCheck[i]) for i in range(len(mmsVars))])
plotFix.opts(logx=True,logy=True,xlabel='$N_x$',ylabel='max($\delta$)',ylim=(1e-9,1))

In [ ]:
hv.save(plotNoFix, 'mmsTestNoFix.pdf', dpi=144)
hv.save(plotFix, 'mmsTestFix.pdf', dpi=144)


### Number of acoustic transition times the simulation is run for

In [ ]:
20000*lengthNorm*np.sqrt(T0*elMass/ionMass)/L